# Testing the GSEA enrichment statistics

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from time import time, process_time

In [5]:
def read_gene_sets(gmt_files):
    # Reads one of more GMT files and returns a dictionary with gene set names as keys
    
    gene_sets = {}

    if not isinstance(gmt_files, list):
        gmt_files = [gmt_files]
    
    for file in gmt_files:

        with open(file) as gsets_file:
            for line in gsets_file:
                line_list = line.strip().split(sep="\t")
                gene_sets[line_list[0]] = {}
                gene_sets[line_list[0]]['description'] = line_list[1]
                gene_sets[line_list[0]]['genes'] = set(line_list[2:])
                gene_sets[line_list[0]]['size'] = len(gene_sets[line_list[0]]['genes'])

    return(gene_sets)

# Call the GSEA enrichment statistics with a simple example
## Gene list with 20 genes and gene set with 6 genes

In [6]:
Coller_et_al_scores = pd.read_csv('../GSEA_Benchmarks_2023/results/Coller_et_al//Coller_et_al_scores.txt', 
                                  index_col = 0, sep = '\t')
Coller_et_al_scores = Coller_et_al_scores.iloc[:, 0]

gene_set = {'MYC', 'IPO4', 'NOL5A', 'EXOSC5', 'IMP4', 'NOL6'}

alpha = 1.0

gene_score2 = Coller_et_al_scores.iloc[0:20]
S_h = gene_score2
gene_list = list(S_h.index)
I_h = pd.Series(0, index = gene_list)
I_h.loc[list(gene_set)] = 1

In [7]:
import pyximport
import numpy as np
pyximport.install(setup_args={"include_dirs":np.get_include(),
                              "extra_compile_args": "-O4"}, inplace=True)
import sys
sys.path.insert(0, '/Users/pablotamayo/UCSD_2022/NEW_GSEA/notebooks')
from compute_KL_PQ_enrichment import compute_KL_PQ_enrichment     
from compute_KL_PRURm_enrichment import compute_KL_PRURm_enrichment  
from compute_KL_PRURp_enrichment import compute_KL_PRURp_enrichment  
from compute_KL_PR_enrichment import compute_KL_PR_enrichment    
from compute_KS_AUC_enrichment import compute_KS_AUC_enrichment    
from compute_KS_SUP_enrichment import compute_KS_SUP_enrichment 

# Timings of gene set's enrichment over the entire gene list

In [8]:
gene_sets = read_gene_sets('../GSEA_Benchmarks_2023/Gene_Sets_Collections/h.all.v2022.1.Hs.symbols.gmt')
gene_set = gene_sets['HALLMARK_MYC_TARGETS_V1']['genes']
gene_list = list(Coller_et_al_scores.index)
gene_set = gene_set.intersection(gene_list)

alpha = 1.0
I_h = pd.Series(0, index = gene_list)
I_h.loc[list(gene_set)] = 1
S_h = Coller_et_al_scores

%timeit Delta, ES = compute_KS_SUP_enrichment(S_h.values, I_h.values, alpha) 
%timeit Delta, ES = compute_KS_AUC_enrichment(S_h.values, I_h.values, alpha) 
%timeit Delta, ES = compute_KL_PR_enrichment(S_h.values, I_h.values, alpha) 
%timeit Delta, ES = compute_KL_PQ_enrichment(S_h.values, I_h.values, alpha) 
%timeit Delta, ES = compute_KL_PRURm_enrichment(S_h.values, I_h.values, alpha) 
%timeit Delta, ES = compute_KL_PRURp_enrichment(S_h.values, I_h.values, alpha) 

45.9 µs ± 825 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
43.8 µs ± 730 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
177 µs ± 520 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
178 µs ± 930 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
310 µs ± 3.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
310 µs ± 2.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Enrichment Scores

In [9]:
Delta, ES = compute_KS_SUP_enrichment(S_h.values, I_h.values, alpha) 
print('KS_SUP: {}'.format(np.round(ES, 4)))
Delta, ES = compute_KS_AUC_enrichment(S_h.values, I_h.values, alpha) 
print('KS_AUC: {}'.format(np.round(ES, 4)))
Delta, ES = compute_KL_PR_enrichment(S_h.values, I_h.values, alpha) 
print('KL_PR: {}'.format(np.round(ES, 4)))
Delta, ES = compute_KL_PQ_enrichment(S_h.values, I_h.values, alpha)
print('KL_PQ: {}'.format(np.round(ES, 4)))
Delta, ES = compute_KL_PRURm_enrichment(S_h.values, I_h.values, alpha) 
print('KL_PRURm: {}'.format(np.round(ES, 4)))
Delta, ES = compute_KL_PRURp_enrichment(S_h.values, I_h.values, alpha) 
print('KL_PRURp: {}'.format(np.round(ES, 4)))

KS_SUP: 0.6823
KS_AUC: 0.3988
KL_PR: 0.7171
KL_PQ: 0.817
KL_PRURm: 0.3643
KL_PRURp: 0.3528
